In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [5]:
df = pd.read_csv('cardekho_dataset.csv')
df.tail(2)

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
15409,19542,Mahindra XUV500,Mahindra,XUV500,5,3800000,Dealer,Diesel,Manual,16.0,2179,140.0,7,1225000
15410,19543,Honda City,Honda,City,2,13000,Dealer,Petrol,Automatic,18.0,1497,117.6,5,1200000


## Data Cleaning
1. Handling Missing Values
2. Handling Duplicates
3. Check Data Type
4. Understand the dataset

In [6]:
# Checking if Null values is present or not
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [7]:
# Dropping Unnecessary Columns

"""
3 columns i.e. car_name, brand and model has similar values that's why removing 2 of them (car_name,brand)
because brand and car_name can be repeated but model is a unique car.
"""

df.drop(columns=['Unnamed: 0','car_name','brand'],axis = 1, inplace=True)

In [8]:
df.head(2)

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.7,796,46.3,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.9,1197,82.0,5,550000


In [9]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [10]:
# Selecting all different types of features

num_features = [feature for feature in df.columns if df[feature].dtype != 'O']
print(f"Number of numerical features is {len(num_features)}")
cat_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print(f"Number of catergorical features is {len(cat_features)}")
discrete_features = [feature for feature in num_features if len(df[feature].unique())<=25]
print(f"Number of discrete features is {len(discrete_features)}")
continous_features = [feature for feature in num_features if feature not in discrete_features]
print(f"Number of continous features is {len(continous_features)}")

Number of numerical features is 7
Number of catergorical features is 4
Number of discrete features is 2
Number of continous features is 5


In [11]:
# Independent & Dependent Features

from sklearn.model_selection import train_test_split
X = df.drop(columns = ['selling_price'], axis = 1)
y = df['selling_price']

## Feature Engineering

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [13]:
# Creating column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type','fuel_type','transmission_type']

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,onehot_columns),
        ("StandardScaler",numeric_transformer,num_features)
    ],remainder='passthrough'
)

In [14]:
X = preprocessor.fit_transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((12328, 17), (3083, 17))

## Model Training & Model Selection

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [20]:
import numpy as np

def evaluate_model(true,predicted):
    """
    Takes true and predicted value as an argument and return metrics
    """
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true,predicted)

    return mae,rmse,r2

In [22]:
models = {
    "Linear Regression":LinearRegression(),
    "Ridge":Ridge(),
    "Lasso":Lasso(),
    "K-Neighbours Regressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regresssor":RandomForestRegressor()
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    
    print("Model Performance for training set")
    print("- Root Mean Squared Error : {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error : {:.4f}".format(model_train_mae))
    print("- R2 Score : {:.4f}".format(model_train_r2))

    print("-----------------------------------------------")

    print("Model Performance for test set")
    print("- Root Mean Squared Error : {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error : {:.4f}".format(model_test_mae))
    print("- R2 Score : {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

Linear Regression
Model Performance for training set
- Root Mean Squared Error : 553880.4331
- Mean Absolute Error : 268213.8787
- R2 Score : 0.6217
-----------------------------------------------
Model Performance for test set
- Root Mean Squared Error : 502519.1133
- Mean Absolute Error : 279830.6101
- R2 Score : 0.6645


Ridge
Model Performance for training set
- Root Mean Squared Error : 553856.3122
- Mean Absolute Error : 268060.0183
- R2 Score : 0.6218
-----------------------------------------------
Model Performance for test set
- Root Mean Squared Error : 502533.3688
- Mean Absolute Error : 279558.1057
- R2 Score : 0.6645


Lasso
Model Performance for training set
- Root Mean Squared Error : 553855.6708
- Mean Absolute Error : 268099.3639
- R2 Score : 0.6218
-----------------------------------------------
Model Performance for test set
- Root Mean Squared Error : 502542.6393
- Mean Absolute Error : 279614.9121
- R2 Score : 0.6645


K-Neighbours Regressor
Model Performance for t

### Hyper-Parameter Tuning
Random Forest Regressor & KNN is performing well so we will do hyper parameter tuning for both of these models

In [26]:
knn_params = {"n_neighbors":[2,3,5,10,20,40,50]}
rf_params = {
    "max_depth":[5,8,15,None,10],
    "max_features":[5,7,"auto",8],
    "min_samples_split":[2,8,15,20],
    "n_estimators":[100,200,500,1000]         
             }

In [27]:
# Model list for hyperparameter tuning
randomcv_models = [
    ("KNN",KNeighborsRegressor(),knn_params),
    ("RF",RandomForestRegressor(),rf_params)
]

In [28]:
# Hyper Parameter Tuning
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                param_distributions=params,
                                n_iter=100,
                                cv=3,
                                verbose=2,
                                n_jobs=-1)
    
    random.fit(X_train,y_train)
    model_param[name] = random.best_params_

    for model_name in model_param:
        print(f"----------------------------------BEST PARAM FOR {model_name}------------------------------------")
        print(model_param[model_name])

Fitting 3 folds for each of 7 candidates, totalling 21 fits
[CV] END ......................................n_neighbors=2; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=2; total time=   0.2s
[CV] END ......................................n_neighbors=5; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.3s
[CV] END ......................................n_neighbors=5; total time=   0.2s
[CV] END ......................................n_neighbors=2; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.3s
[CV] END ......................................n_neighbors=5; total time=   0.2s
[CV] END .....................................n_neighbors=10; total time=   0.2s
[CV] END .....................................n_neighbors=10; total time=   0.2s
[CV] END .....................................n_n

In [30]:
"""
----------------------------------BEST PARAM FOR KNN------------------------------------
{'n_neighbors': 5}
----------------------------------BEST PARAM FOR RF------------------------------------
{'n_estimators': 1000, 'min_samples_split': 2, 'max_features': 5, 'max_depth': 15}

This is our best parameters for KNN and Random Forest. Now we will use this to train our model
"""

# Re-Training the models with best parameters

models = {
    "Random Forest Regressor": RandomForestRegressor(n_estimators= 1000, min_samples_split= 2, max_features= 5, max_depth= 15),
    "K-Neighbours Regressor": KNeighborsRegressor(n_neighbors=5)
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    
    print("Model Performance for training set")
    print("- Root Mean Squared Error : {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error : {:.4f}".format(model_train_mae))
    print("- R2 Score : {:.4f}".format(model_train_r2))

    print("-----------------------------------------------")

    print("Model Performance for test set")
    print("- Root Mean Squared Error : {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error : {:.4f}".format(model_test_mae))
    print("- R2 Score : {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

Random Forest Regressor
Model Performance for training set
- Root Mean Squared Error : 140810.2384
- Mean Absolute Error : 57073.6160
- R2 Score : 0.9756
-----------------------------------------------
Model Performance for test set
- Root Mean Squared Error : 207083.6618
- Mean Absolute Error : 97863.1654
- R2 Score : 0.9430


K-Neighbours Regressor
Model Performance for training set
- Root Mean Squared Error : 327413.8524
- Mean Absolute Error : 92348.6210
- R2 Score : 0.8678
-----------------------------------------------
Model Performance for test set
- Root Mean Squared Error : 255104.1047
- Mean Absolute Error : 113605.9439
- R2 Score : 0.9135


